### Investigate max durations outliers in item and student attempt times
The duration between student start and stop times have large number of outliers of over 100 minutes

In [19]:
%run '../lib/libraries.ipynb'

dython              0.6.1
tinydb              4.2.0


You should consider upgrading via the '/Users/duane.hinkley/PycharmProjects/c772-capstone-project/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/duane.hinkley/PycharmProjects/c772-capstone-project/venv/bin/python -m pip install --upgrade pip' command.


In [20]:
filterDf = load_df('filterDf')
types = get_var_types()

#### Create dataframe with durations

###### Count before

In [21]:
filterDf.count()

80548

- 80,548

In [22]:
durDf = add_attempt_duration(filterDf)

###### Count after

In [23]:
durDf.count()

80548

- 80,548

###### Remove stop dates before start dates to focus on max

In [24]:
remove_stop_dates_before_start_dates(durDf).count()

65541

- removed too many
  - ~ 15K observations

##### Investigate removing too many dates


###### Preview an assessment with outliers (assessment_instance_attempt_id == 770557)

In [25]:
cols = [
       'number_of_distinct_instance_items',
       'assignment_late_submission',
       'assignment_final_submission_date',
       'assignment_start_date',
       'assignment_due_date',
       'was_fully_scored_datetime',
       'was_submitted_datetime_actual',
       'was_in_progress_datetime',
       'is_force_scored',
       'is_manual_scoring_required',
       'student_start_datetime', 
       'student_stop_datetime', 
       'item_is_offline_scored',
       'learner_attempt_status',
       'points_possible_unweighted',
       'final_score_unweighted',
       'scored_datetime',    
       'item_type_code_name',
       'scoring_type_code',
       'raw_score',
       'response_correctness',
       'item_attempt_start_datetime_utc',
       'item_attempt_end_datetime_utc',
       'assigned_item_status',
       'assessment_item_response_id',
       
]
filterDf.filter( F.col('assessment_instance_attempt_id') == 770557).show(50, False)

+------------------------+------------------+------------------+---------------------------------+-----------------------+--------------------------+--------------------------------+---------------------+-------------------+--------------------------+-------------------------+-------------------------+-------------------------+-----------------------------+------------------------+---------------+--------------------------+-----------------------+----------------------+----------------------+----------------------+--------------------------+----------------------+-------------------+-------------------+-----------------+---------+--------------------+-------------------------------+-----------------------------+--------------------+----------+------+----------+-------------+----------------------+-----------------------------+------------------------------+----------+--------------------------------+---------------------------+
|ced_assignment_type_code|is_affecting_grade|number_of_lea

In [26]:
filterDf.filter( F.col('assessment_instance_attempt_id') == 770557).count()

30

- 30 observations in attempt

###### Save attempt dataframe

In [27]:
attDf = filterDf.filter( F.col('assessment_instance_attempt_id') == 770557)
attDf.count()

30

- Still 30 observations

###### item start date <= stop date

In [28]:
attDf.filter(
       ( F.col('item_attempt_start_datetime_utc') <= F.col('item_attempt_end_datetime_utc') )
   ).count()

18

- 18

###### item start date > stop date

In [29]:
attDf.filter(
       ( F.col('item_attempt_start_datetime_utc') > F.col('item_attempt_end_datetime_utc') )
   ).count()

0

- 0

###### item start date or stop date null

In [30]:
attDf.filter(
       ( F.col('item_attempt_start_datetime_utc').isNull() | F.col('item_attempt_end_datetime_utc').isNull() )
   ).count()

12

- 12
  - 12 + 18 = 30

###### item start date or stop date null using filter

In [31]:
clean_item_attempt_dates(attDf).count()

18

- same 18 dates

###### full dataset item start date or stop date null

In [32]:
filterDf.filter(
       ( F.col('item_attempt_start_datetime_utc').isNull() | F.col('item_attempt_end_datetime_utc').isNull() )
   ).count()


14999

The 14999 removed was null dates


##### Statistics on full set

In [ ]:
durDf.select(types['durationVars']).toPandas().describe()

##### Statistics on not null dates

,item_attempt_duration_mins,student_duration_mins
count,65549.000000,79799.000000
mean,523.712277,1506.039395
std,3453.739951,7443.391797
min,-54766.016667,-1362.766667
25%,0.416667,4.866667
50%,1.800000,22.533333
75%,9.533333,90.566667
max,109353.100000,261008.166667


###### item start date > stop date

In [34]:
durDf.filter(
       ( F.col('student_stop_datetime') <= F.col('student_stop_datetime') )
   ).select( types['durationVars'] ).toPandas().describe()


,item_attempt_duration_mins,student_duration_mins
count,65104.000000,79799.000000
mean,525.695223,1506.039395
std,3463.177723,7443.391797
min,-54766.016667,-1362.766667
25%,0.416667,4.866667
50%,1.816667,22.533333
75%,9.583333,90.566667
max,109353.100000,261008.166667


In [ ]:
durDf.filter( F.col('assessment_instance_attempt_id') == 770557).show(50, False)

##### offline_scored has null item_attempt_start_datetime_utc, item_attempt_end_datetime_utc and assessment_item_response_id

In [ ]:
finish_todo('Investigate max durations outliers in item and student attempt times')